### Implement RandomizedMotifSearch.

Input: Integers k and t, followed by a space-separated collection of strings Dna.

Output: A collection BestMotifs resulting from running RandomizedMotifSearch(Dna, k, t) 1,000 times. Remember to use pseudocounts!

In [106]:
import random
import pandas as pd
import time
import numpy as np

In [67]:
def mostProbKmer(text, k, profile):
    #process profile into usable format
    profDict = {}
    profDict['A'] = profile[0]
    profDict['C'] = profile[1]
    profDict['G'] = profile[2]
    profDict['T'] = profile[3]
    #print(profDict)
    probsDict = {}
    
    for i in range(0, len(text)-k+1):
        pat = text[i:k+i]
        prob = 1
        for b in range(0, len(pat)): #len(pat)=k
            prob*=float(profDict[pat[b]][b])
        probsDict[pat] = prob
    #print(probsDict)
    for key, val in probsDict.items():
        if val == max(probsDict.values()):
            return key
            

In [92]:
def score(dnas):
    score = 0
    dnasDF = pd.DataFrame(dnas)[0].str.split('',expand=True).iloc[:,1:-1]

    for i in range(0,dnasDF.shape[1]):
        valCts = dnasDF.iloc[:,i].value_counts()
        score+=(len(dnas)-max(valCts))    
    return score

In [125]:
# profile
def profile(motifs):
    k = len(motifs[0])
    dicNuc = {'A':0, 'C':1, 'G':2, 'T':3}
    profCts = np.array([[1]*k,
                            [1]*k,
                            [1]*k,
                            [1]*k])
    
    profile = np.array([[float(0)]*k,
                        [float(0)]*k,
                        [float(0)]*k,
                        [float(0)]*k])
    
    for n in range (0, k): 
        for motf in motifs:
            #get profile counts
            profIndx = dicNuc[motf[n]]
            profCts[profIndx,n]+=1 
        totCol = profCts[:,n].sum()     
        for r in range (0,4):
            profile[r, n] = profCts[r, n]/totCol
        
    return profile                

In [93]:
def randomizedMotifSearch(dna, k, t):
    motifs = []
    for stri in dna:
        # random motifs
        r = random.randrange(0,len(stri)-k+1)
        #print(r)
        motifs.append(stri[r:k+r])
    bestMotifs = motifs
    
    while 1==1:
        prof = profile(motifs)
        motifs = []
        for stri in dna:    
            motifs.append(mostProbKmer(stri, k , prof))
        
        scor = score(motifs)
        bestScore = scor
        if scor < score(bestMotifs):
            bestMotifs = motifs
            bestScore = scor
        else:
            return bestScore, bestMotifs

In [94]:
file = pd.read_csv("dataset_30307_5 (1).txt", header=None)
print(file.iloc[1,0].split()[1])

CCATGCTAAGTCCAATCTAGCGGTCTTGAGTACCCGGGAAAAATGACAGTATACTTACTCGGCCCGGAAACGAGAGGGTCAAGCAGTTGTTTGCTGTCAGTTGGGGTTAACAGTCCGCAAGGATCCCAATCCCTTGCGCGCTCTAGAAACGTTCATCGGATATTTCGAGAATACTCTTGATTTCAGGAGCGCCATTTAAGAACC


In [96]:
# RUNNING CELL
startt = time.time()
motDi = {}
for i in range (0,1000):
    i+=1
    key, value = randomizedMotifSearch(pd.Series(file.iloc[1,0].split()), int(file.iloc[0,0].split()[0]),int(file.iloc[0,0].split()[1]))
    motDi[key] = value
        
for key, val in motDi.items():
    if key == min(motDi.keys()):
        print("--- Running time: %s seconds ---" % (time.time() - startt))
        print(*val)

--- Running time: 156.08359217643738 seconds ---
TTGATTTCAGGAGCG CCGGAAACGAGAGGG CCGATTAATGGAGGG CCGATTACGCCGGGG GCGATTACGAGAGTA CCGATGGGGAGAGGG CAATTTACGAGAGGG CCACGTACGAGAGGG TGAATTACGAGAGGG TTGATTACGAGAGGC CCGATGGTGAGAGGG CCGATTTAAAGAGGG CCGATTACCTCAGGG CCGGACACGAGAGGG CCGAAGCCGAGAGGG CCGATTACGAGCTTG CCGATTACGATGTGG CCGATTTGAAGAGGG CCGACGCCGAGAGGG CCGATTACGAGACAC


Implement the random number generator Random(p1, …, pn) so that it uses RandomNumber(X) (for an appropriately chosen integer X) as a subroutine

In [196]:
def randomWeighted(p):
    return random.choices([i for i in range(0,len(p))], weights= p, k= 1)[0]

In [197]:
randomWeighted([0.1, 0.2, 0.3, 0.05, 0.1, 0.25])

4

### Implement GibbsSampler

Input: Integers k, t, and N, followed by a space-separated collection of strings Dna.

Output: The strings BestMotifs resulting from running GibbsSampler(Dna, k, t, N) with 20 random starts. Remember to use pseudocounts!


In [171]:
def kmerProbabilities(text, k, profile):
    #process profile into usable format
    profDict = {}
    profDict['A'] = profile[0]
    profDict['C'] = profile[1]
    profDict['G'] = profile[2]
    profDict['T'] = profile[3]
    #print(profDict)
    probsDict = {}
    
    for i in range(0, len(text)-k+1):
        pat = text[i:k+i]
        prob = 1
        for b in range(0, len(pat)): #len(pat)=k
            prob*=float(profDict[pat[b]][b])
        probsDict[pat] = prob
    #print(probsDict)
    
    return probsDict
            

In [438]:
# without 20 random starts --> preferred (time)
def gibbsSampler(dna, k, t, N):
    startt = time.time()
    motifs = []
    for stri in dna:
        # random motifs
        r = random.randrange(0,len(stri)-k+1)
        motifs.append(stri[r:k+r])
    #print(motifs)
    bestMotifs = motifs
    
    for n in range (0,N):
        i = random.randrange(0,t)
        #print(i)
        profNotI = profile(motifs[:i]+motifs[i+1:])
        #print(profNotI)
        probsDict = kmerProbabilities(dna[i], k, profNotI)
        #print(probsDict) 
        probs = list(probsDict.values())
        kmers = list(probsDict.keys())
        #print(probsDict.values())
        randIndex = randomWeighted(probs)
        #print(randIndex)
    
        motifs = motifs[:i]+[kmers[randIndex]]+motifs[i+1:]

        bestScore = score(bestMotifs)
        currScore = score(motifs)
        if currScore<bestScore:
            #print(currScore)
            bestMotifs= motifs
    #print("--- Running time: %s seconds ---" % (time.time() - startt))
    #print(*bestMotifs)   
    return bestScore, bestMotifs

In [411]:
for i in range(0,10):
    dna = pd.Series(["CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA", "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG", "TAGTACCGAGACCGAAAGAAGTATACAGGCGT", "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC", "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"])
    print(gibbsSampler(dna, 8,5,2000))

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['TCAGTAAC', 'CAAGGTGC', 'ACAGGCGT', 'TCAGGTGC', 'CCACGTGC']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['GGGTGTTC', 'ATGTGTAA', 'AAGTATAC', 'AGGTGCAC', 'ACGTGCAA']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']


In [431]:
file = pd.read_csv("dataset_30309_11 (2).txt", header=None)
print(file.iloc[0,0].split()[2])

2000


In [433]:
gibbsSampler(pd.Series(file.iloc[1,0].split()),
    int(file.iloc[0,0].split()[0]), int(file.iloc[0,0].split()[1]), 10000)

TTTGTTGGCTTTCTT TATCCCTGCTTCCTG TATTTTTGCTTCACT TATTACCGCTTCCTG TATTTTGTGTTCCTG TCGGTTTGCTTCCTG TATTTTCCGTTCCTG CGTTTTTGCTTCCTT TATTTTTCGCTCCTG TATTTTTGGGCCCTG TAGGATTGCTTCCTG TATTTGATCTTCCTG TATTTAGTCTTCCTG TATTTTTGCTTATCG TATAGATGCTTCCTG GATTTTTGCTTCCAA TATTTTTGCTAGATG TATTTTTGCGAGCTG TATTAGCGCTTCCTG CCGTTTTGCTTCCTG


In [418]:
out = gibbsSampler(pd.Series(file.iloc[1,0].split()),
    int(file.iloc[0,0].split()[0]), int(file.iloc[0,0].split()[1]), 2000)
out

['ACGTCCACCGGCGTC',
 'AAGCGCACCGGGGTG',
 'ACCCTTACCGGGGTG',
 'AAGTTCCTCGGGGTG',
 'AAGTTTTATGGGGTG',
 'AAGTTTACCGGGTGC',
 'AAGTTTCGAGGGGTG',
 'CTGTTTACCGGGGTA',
 'AAGTTGCTCGGGGTG',
 'AAACATACCGGGGTG',
 'AAGTTTAGGAGGGTG',
 'AAGGAAACCGGGGTG',
 'AAGTTTACACAGGTG',
 'TAGTTTACCGGGGAT',
 'CCTTTTACCGGGGTG',
 'AAGTGAGCCGGGGTG',
 'AAGTCGTCCGGGGTG',
 'AAGTTTACCGGACAG',
 'AAGTTTACCAATGTG',
 'AAGTTTACCGTCATG']

In [420]:
score(out)

62

In [372]:
theanswer = list(pd.read_csv("GibbsSampler/outputs/output_2.txt", header=None)[0])[0].split()
theanswer

['ACGTCCACCGGCGTC',
 'AAGCGCACCGGGGTG',
 'ACCCTTACCGGGGTG',
 'AAGTTCCTCGGGGTG',
 'AAGTTTTATGGGGTG',
 'AAGTTTACCGGGTGC',
 'AAGTTTCGAGGGGTG',
 'CTGTTTACCGGGGTA',
 'AAGTTGCTCGGGGTG',
 'AAACATACCGGGGTG',
 'AAGTTTAGGAGGGTG',
 'AAGGAAACCGGGGTG',
 'AAGTTTACACAGGTG',
 'TAGTTTACCGGGGAT',
 'CCTTTTACCGGGGTG',
 'AAGTGAGCCGGGGTG',
 'AAGTCGTCCGGGGTG',
 'AAGTTTACCGGACAG',
 'AAGTTTACCAATGTG',
 'AAGTTTACCGTCATG']

In [373]:
score(theanswer)

62

In [422]:
out==theanswer

True

In [448]:
# Implement 20 random starts --> makes it too slow!
def gibbsSampler_sRandStrts (dna, k, t, N, s):
    startt = time.time()
    bestScore = float(inf)
    
    for i in range(s):
        currScore, motifs = gibbsSampler(dna, k, t, N)
        if currScore<bestScore:
            #print(currScore)
            bestScore = currScore
            bestMotifs = motifs
    
    #print(bestScore)
    print(*bestMotifs)
    print("--- Running time: %s seconds ---" % (time.time() - startt))

In [447]:
gibbsSampler_sRandStrts(dna, 8,5,100,20)

13
9
9
TCTCGGGG CCAAGGTG TACAGGCG TTCAGGTG TCCACGTG
--- Running time: 28.55189299583435 seconds ---


In [433]:
gibbsSampler_sRandStrts(pd.Series(file.iloc[1,0].split()),
    int(file.iloc[0,0].split()[0]), int(file.iloc[0,0].split()[1]), int(file.iloc[0,0].split()[2]), 20)

TTTGTTGGCTTTCTT TATCCCTGCTTCCTG TATTTTTGCTTCACT TATTACCGCTTCCTG TATTTTGTGTTCCTG TCGGTTTGCTTCCTG TATTTTCCGTTCCTG CGTTTTTGCTTCCTT TATTTTTCGCTCCTG TATTTTTGGGCCCTG TAGGATTGCTTCCTG TATTTGATCTTCCTG TATTTAGTCTTCCTG TATTTTTGCTTATCG TATAGATGCTTCCTG GATTTTTGCTTCCAA TATTTTTGCTAGATG TATTTTTGCGAGCTG TATTAGCGCTTCCTG CCGTTTTGCTTCCTG


In [449]:
gibbsSampler_sRandStrts(pd.Series(file.iloc[1,0].split()),
    int(file.iloc[0,0].split()[0]), int(file.iloc[0,0].split()[1]), int(file.iloc[0,0].split()[2]), 20)

TTTGTTGGCTTTCTT TATCCCTGCTTCCTG TATTTTTGCTTCACT TATTACCGCTTCCTG TATTTTGTGTTCCTG TCGGTTTGCTTCCTG TATTTTCCGTTCCTG CGTTTTTGCTTCCTT TATTTTTCGCTCCTG TATTTTTGGGCCCTG TAGGATTGCTTCCTG TATTTGATCTTCCTG TATTTAGTCTTCCTG TATTTTTGCTTATCG TATAGATGCTTCCTG GATTTTTGCTTCCAA TATTTTTGCTAGATG TATTTTTGCGAGCTG TATTAGCGCTTCCTG CCGTTTTGCTTCCTG
--- Running time: 820.8851959705353 seconds ---
